In [60]:
# train_spell_error.txt
# train_clean.txt
spell_error = open("train_spell_error.txt", "r")
clean = open("train_clean.txt", "r")
spell_error = spell_error.read().split("\n")
clean = clean.read().split("\n")

def letterSim(A,B):
    tedict = {}
    for i in range(len(A)):
        if A[i] not in tedict:
            tedict[A[i]] = 1
        else:
            tedict[A[i]] += 1
    for i in range(len(B)):
        if B[i] not in tedict:
            tedict[B[i]] = -1
        else:
            tedict[B[i]] -= 1
    sum = 0
    for i in tedict:
        sum += abs(tedict[i])
    to = len(A) + len(B)
    to = to / 2
    return to,sum

# 将spell_error和clean的数据每个句子以空格切分，视为单词，每个单词最后一个元素为“:”则将其单独切分
for i in range(len(spell_error)):
    temp = spell_error[i].split(" ")
    spell_error[i] = []
    for j in range(len(temp)):
        # print(temp)
        # print('*',temp[j],'*')
        if len(temp[j]) == 0:
            continue
        if temp[j][-1] == ":":
            spell_error[i].append(temp[j][:-1])
            spell_error[i].append(":")
        else:
            spell_error[i].append(temp[j])

for i in range(len(clean)):
    temp = clean[i].split(" ")
    clean[i] = []
    for j in range(len(temp)):
        if len(temp[j]) == 0:
            continue
        if temp[j][-1] == ":":
            clean[i].append(temp[j][:-1])
            clean[i].append(":")
        else:
            clean[i].append(temp[j])

for i in range(len(spell_error)):
    if len(spell_error[i]) != len(clean[i]):
        for j in range(len(spell_error[i])):
            # 如果含有“:”且不是最后一个元素
            if spell_error[i][j].find(":") != -1 and spell_error[i][j][-1] != ":":
                # spell_error[i][j]位置赋予删除“:”后的单词
                spell_error[i][j] = spell_error[i][j].replace(":", "")
                # spell_error[i][j]之后插入“:”
                spell_error[i].insert(j+1, ":")


OUT = {}
# 错误绝大部分都是对应位置的单词的拼写错误 
ta = {}
tb = {}
errerDict = {}
norDict = {}
tol = 0  #79791
onesenhavemulerrors = 0   #22883
senlenserr = 0  #34
for i in range(len(spell_error)):
    indexx = 0
    for  j in range(len(spell_error[i])):
        if len(spell_error[i]) == len(clean[i]) and spell_error[i][j] != clean[i][j]:
            tol += 1
            indexx += 1

        #     if spell_error[i][j] not in errerDict:
        #         errerDict[spell_error[i][j]] = 1
        #     else:
        #         errerDict[spell_error[i][j]] += 1

        #     if clean[i][j] not in norDict:
        #         norDict[clean[i][j]] = 1
        #     else:
        #         norDict[clean[i][j]] += 1
        # else:
        #     if clean[i][j] not in norDict:
        #         norDict[clean[i][j]] = 1
        #     else:
        #         norDict[clean[i][j]] += 1
        


            # if len(spell_error[i][j]) == 1:
            #     print(spell_error[i][j],clean[i][j])
            #     if spell_error[i][j] not in ta:
            #         ta[spell_error[i][j]] = 1
            #     else:
            #         ta[spell_error[i][j]] += 1

            # if len(clean[i][j]) == 1:
            #     print(spell_error[i][j],clean[i][j])
            #     if clean[i][j] not in tb:
            #         tb[clean[i][j]] = 1
            #     else:
            #         tb[clean[i][j]] += 1

            # a,b = letterSim(clean[i][j],spell_error[i][j])
            # if b > a :
            #     print('#',clean[i][j],'#',spell_error[i][j],'#',letterSim(clean[i][j],spell_error[i][j]))


            # print(spell_error[i])
            # print(clean[i])
            # print(" ".join(clean[i]))
            # print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    if len(spell_error[i]) != len(clean[i]):
        # print(spell_error[i])
        # print(clean[i])
        # print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        senlenserr +=1 
    # if indexx > 1:
        # onesenhavemulerrors += 1



# # 寻找所有":"之前的单词  
# te = {}
# for i in range(len(clean)):
#     for j in range(len(clean[i])-1):
#         if clean[i][j+1] == ":":
#             if clean[i][j] not in te:
#                 te[clean[i][j]] = 1
#             else:
#                 te[clean[i][j]] += 1

# for i in range(len(clean)):
#     for j in range(len(clean[i])):
#         if clean[i][j] in te and len(clean[i]) > j+1:
#             print(clean[i][j+1])
# 单字错误情况
# я	8	——t
# ю	1	
# ю	97	——t
# э	1	
# э	72	——t
# ь	258	——t
# ы	1	
# ы	10	——t
# щ	22	——t
# ш	6	
# ш	23	——t
# ч	300	
# ч	26	——t
# ц	8	
# ц	13	——t
# х	4	
# х	83	——t
# ф	3	
# ф	18	——t
# ү	59	——t
# у	1	
# у	64	——t
# т	10	
# т	37	——t
# с	13	
# с	46	——t
# р	123	
# р	44	——t
# п	2	
# п	14	——t
# ө	6	
# ө	4	——t
# о	2	
# о	7	——t
# н	9	
# н	305	——t
# м	8	
# м	121	——t
# л	221	
# л	28	——t
# к	2	
# к	16	——t
# й	35	——t
# и	3	
# и	115	——t
# з	2	
# з	29	——t
# ж	4	
# ж	38	——t
# ё	6	——t
# е	2	
# е	16	——t
# д	50	
# д	28	——t
# г	22	
# г	49	——t
# в	6	
# в	37	——t
# б	13	
# б	102	——t
# а	3	
# v	2	
# s	1	
# n	1	
# i	1	
# 5	2	


In [61]:
# wordsDict.pickle  读取
import pickle
with open('wordsDict.pickle', 'rb') as f:
    wordsDict = pickle.load(f)
    

In [65]:

norDict
len(errerDict)

56962

In [ ]:
errrepeat = 0  #9578个词是句子中错误拼写但是在正确的词典中
for i in errerDict:
    if i in norDict:
        errrepeat += errerDict[i]
        print(i,errerDict[i],norDict[i])

In [ ]:
errrepeat = 0  #41968 个词是句子中错误拼写但是在正确的词典中 这次是更大词典
for i in errerDict:
    if i not in wordsDict:
        print(i,errerDict[i])
        errrepeat += 1

In [72]:
errrepeat

41968

In [73]:
len(errerDict)

56962

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-large-mongolian-cased', use_fast=False)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-large-mongolian-cased')

## declare task ##
pipe = pipeline(task="fill-mask", model=model, tokenizer=tokenizer)

## example ##
input_  = 'Монгол улсын [MASK] Улаанбаатар хотоос ярьж байна.'

output_ = pipe(input_)
for i in range(len(output_)):
    print(output_[i])

## output ##
# {'sequence': 'Монгол улсын нийслэл Улаанбаатар хотоос ярьж байна.', 'score': 0.9779232740402222, 'token': 1176, 'token_str': 'нийслэл'}
# {'sequence': 'Монгол улсын Нийслэл Улаанбаатар хотоос ярьж байна.', 'score': 0.015034765936434269, 'token': 4059, 'token_str': 'Нийслэл'}
# {'sequence': 'Монгол улсын Ерөнхийлөгч Улаанбаатар хотоос ярьж байна.', 'score': 0.0021413620561361313, 'token': 325, 'token_str': 'Ерөнхийлөгч'}
# {'sequence': 'Монгол улсын ерөнхийлөгч Улаанбаатар хотоос ярьж байна.', 'score': 0.0008035294013097882, 'token': 1215, 'token_str': 'ерөнхийлөгч'}
# {'sequence': 'Монгол улсын нийслэлийн Улаанбаатар хотоос ярьж байна.', 'score': 0.0006434018723666668, 'token': 356, 'token_str': 'нийслэлийн'}




tokenizer_config.json:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

c:\Users\quezi\.conda\envs\weibodatacleaning\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\quezi\.cache\huggingface\hub\models--tugstugi--bert-large-mongolian-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

{'score': 0.9779230356216431, 'token': 1176, 'token_str': 'нийслэл', 'sequence': 'Монгол улсын нийслэл Улаанбаатар хотоос ярьж байна.'}
{'score': 0.015034747309982777, 'token': 4059, 'token_str': 'Нийслэл', 'sequence': 'Монгол улсын Нийслэл Улаанбаатар хотоос ярьж байна.'}
{'score': 0.002141402568668127, 'token': 325, 'token_str': 'Ерөнхийлөгч', 'sequence': 'Монгол улсын Ерөнхийлөгч Улаанбаатар хотоос ярьж байна.'}
{'score': 0.0008035437786020339, 'token': 1215, 'token_str': 'ерөнхийлөгч', 'sequence': 'Монгол улсын ерөнхийлөгч Улаанбаатар хотоос ярьж байна.'}
{'score': 0.0006434023380279541, 'token': 356, 'token_str': 'нийслэлийн', 'sequence': 'Монгол улсын нийслэлийн Улаанбаатар хотоос ярьж байна.'}


In [ ]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/albert-mongolian")
model = AutoModelForMaskedLM.from_pretrained("bayartsogt/albert-mongolian")

########################################################################################################

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/mongolian-roberta-large")
model = AutoModelForMaskedLM.from_pretrained("bayartsogt/mongolian-roberta-large")

########################################################################################################

########################################################################################
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-base-mongolian-uncased', use_fast=False)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-base-mongolian-uncased')

## declare task ##
pipe = pipeline(task="fill-mask", model=model, tokenizer=tokenizer)

## example ##
input_  = 'Миний [MASK] хоол идэх нь тун чухал.'

output_ = pipe(input_)
for i in range(len(output_)):
    print(output_[i])

## output ##
#{'sequence': 'миний хувьд хоол идэх нь тун чухал.', 'score': 0.7889143824577332, 'token': 126, 'token_str': 'хувьд'}
#{'sequence': 'миний бодлоор хоол идэх нь тун чухал.', 'score': 0.18616807460784912, 'token': 6106, 'token_str': 'бодлоор'}
#{'sequence': 'миний зүгээс хоол идэх нь тун чухал.', 'score': 0.004825591575354338, 'token': 761, 'token_str': 'зүгээс'}
#{'sequence': 'миний биед хоол идэх нь тун чухал.', 'score': 0.0015743684489279985, 'token': 3010, 'token_str': 'биед'}
#{'sequence': 'миний тухайд хоол идэх нь тун чухал.', 'score': 0.0014919431414455175, 'token': 1712, 'token_str': 'тухайд'}


In [1]:
#!/usr/bin/env python3
"""Download and pre process the Mongolian Wikipedia."""
__author__ = 'Erdene-Ochir Tuguldur'

import os
import glob
from os.path import join, exists
from utils import download_file, sentence_tokenize

MN_WIKI_FILE = 'mn_wiki.txt'
MN_CORPUS_FOLDER = 'mn_corpus'
MN_WIKI_RAR_FILE = 'mn_wiki.bz2'
MN_WIKI_EXTRACT_FOLDER = 'tmp_mn_wiki'
MN_WIKI_URL = 'https://dumps.wikimedia.org/mnwiki/20181220/mnwiki-20181220-pages-articles-multistream.xml.bz2'


# create corpus directory
if not exists(MN_CORPUS_FOLDER):
    os.makedirs(MN_CORPUS_FOLDER)

# download
if not exists(MN_WIKI_RAR_FILE):
    download_file(MN_WIKI_URL, MN_WIKI_RAR_FILE)

# extract
os.system("python3 wikiextractor/WikiExtractor.py %s -o=%s" % (MN_WIKI_RAR_FILE, MN_WIKI_EXTRACT_FOLDER))


def _pre_process(wiki_file_name):
    """a very simple pre processing"""
    print("pre processing '%s'..." % wiki_file_name)
    with open(wiki_file_name) as f:
        content = f.readlines()

    # some articles has only 1 line, so write first into an array later filter them
    articles = [[]]

    article_start = False
    # process line by line
    for line in content:
        line = line.strip()

        # ignore empty line
        if not line:
            continue
        # ignore article start
        if line.startswith('<doc'):
            article_start = True
            continue
        # after article start, there is the article title, ignore it
        if article_start:
            article_start = False
            continue
        # ignore categories
        if line.startswith('[['):
            continue
        if line.startswith('</doc>'):
            articles.append([])
            continue
        # tokenize the news article sentence by sentence
        for sentence in sentence_tokenize(line):
            articles[-1].append(sentence)

    # we need articles with at least 2 lines
    articles = [d for d in articles if len(d) >= 2]

    # write into a single file
    with open(join(MN_CORPUS_FOLDER, MN_WIKI_FILE), 'a') as f:
        for article in articles:
            for sentence in article:
                f.write(sentence)
                f.write('\n')
            # extra new line to separate each news articles
            f.write('\n')


# pre process the extracted news files
print('%s/*/*.txt' % MN_WIKI_EXTRACT_FOLDER)
for file_name in glob.glob('%s/*/wiki_*' % MN_WIKI_EXTRACT_FOLDER):
    _pre_process(file_name)
print("done!")

ImportError: cannot import name 'download_file' from 'utils' (c:\Users\quezi\.conda\envs\weibodatacleaning\lib\site-packages\utils\__init__.py)